In [ ]:
# standard imports
import numpy as np
import math
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh
from derivatives import finite_differences,polynomial_derivatives
from derivatives import bilinear_derivatives
from isentropic_navier_stokes import Q,L,inner_product

# utility functions
from utilities_modified import *

# Setting Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# define mid point for rotational symmetric mesh
mid_point = np.array([0.5,0]) 
# convert mesh into custom class exp_mesh
mesh = exp_mesh(meshio_mesh,mid_point)

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_comp_air/mach 0.01/"
data_dict = read_csv_data(data_dir,delay=5000,subsampling=1,max_size=400) 

# Loading Filedata

In [ ]:
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]
u = np.divide(data_dict['Momentum_x'],data_dict['Density'])
v = np.divide(data_dict['Momentum_y'],data_dict['Density'])
a = data_dict['Mach']
n = len(x)

# Geometric Constants

In [ ]:
# DEFINING CONSTANTS
n = mesh.n
N = mesh.N
alpha = 1 / 1.4
method="fd"
gamma = 1.4

In [ ]:
# node coordinates
x_node = mesh.points[:,0]
y_node = mesh.points[:,1]

# cell center coordinates
x_cell = np.empty(N)
y_cell = np.empty(N)

for cel in mesh.cells:
    i = cel.index
    [x_cell[i],y_cell[i]] = cel.center

# POD Decomposition

In [ ]:
q = np.vstack([u,v,a])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)
q_fluc = q

In [ ]:
U,S,Vh =  np.linalg.svd(q_fluc,full_matrices=False)

epsilon = 0.97 # [0,1]
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1

num = 5

In [ ]:
pod_modes = U

In [ ]:
# incorrect because POD modes are not orthonormal in euclidean space but in problem space
coeffs = np.matmul(np.linalg.pinv(pod_modes),q_fluc)

In [ ]:
plot_activations(coeffs,num)

## Galerkin System

In [ ]:
method = "fd"
output = "node"
if output=="cell":
    dim = N
else:
    dim = n


# initialie arrays for parameters
L1 = np.empty((num,num))
L2 = np.empty((num,num))
Q_ = np.array([np.empty((num,num)) for _ in range(num)])

# pod basis for computation
if output=="cell":
    modes = np.empty((3*N,num))
    for i in range(num):
        modes[:N,i] = mesh.compute_cell_values_from_node_data(pod_modes[:n,i])
        modes[N:2*N,i] = mesh.compute_cell_values_from_node_data(pod_modes[n:2*n,i])
        modes[2*N:3*N,i] = mesh.compute_cell_values_from_node_data(pod_modes[2*n:3*n,i])
else:
    modes = pod_modes


# compute L and Q operators for projectionW
Q_tmp = np.empty((num,num,3*dim))
L_tmp = np.empty((num,3*dim))

for i in tqdm(range(num)):
    L_tmp[i] = L(mesh,pod_modes[:,i],method=method,output=output)
    for j in range(num):
        Q_tmp[i,j,:] = Q(mesh,pod_modes[:,i],pod_modes[:,j],method=method,output=output)

# compute ODE coefficients
L1 = np.matmul(L_tmp,modes[:,:num])
for i in range(num):
    for j in range(num):
        Q_[i,j,:] = np.matmul(modes[:,:num].T,Q_tmp[i,j])

# Solving ODE system

In [ ]:
# initial conditions for the activations are obtained as activations on the initial timeframe of the data matrix
a0 = coeffs[:num,0] #initial conditions
dt = 0.1
t0 = 0
tmax = 100
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))

# static viscosity
nu = 1.516e-5

# defining ode function for node based coefficients
def galerkin_system(t,a):
    # set global variables references
    global Q_, L1, nu, num
    
    a = np.expand_dims(a,1)
    a_dot = np.empty_like(a)
    
    quad = np.zeros((num,1))

    for k in range(a_dot.shape[0]):
        quad[k] = np.matmul(np.matmul(a.T,np.array(Q_)[:,:,k]),a)  
    a_dot = np.matmul((nu*L1),a) + quad
    return a_dot.squeeze()

In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol

In [ ]:
plot_activations(sol.y,num)